In [5]:
import pandas as pd

## Load Data

Documentation: https://mover.ics.uci.edu/documentation.html

### Data files

- `patient_information`:
    1. demographic information including age, sex, height, and weight,
    2. information regarding the surgery being performed including: the type of surgery, the start and end time of both the surgery and anesthesia, ASA status, and discharge disposition.


- `patient_labs`: labs ordered for a patient, the corresponding observed measurements, and the reference measurements for each lab.

- `patient_procedure events`: preoperative, perioperative, and postoperative procedural events and the corresponding time of the event.

- `patient_post_op_complications`: the type of complication, when the complication was observed, and a free text note field outlining more specific details

Merge by `MRN` and `LOG_ID`.


In [6]:
# load data
pat_info = pd.read_csv("Data/patient_information.csv")
pat_labs = pd.read_csv("Data/patient_labs.csv")
pat_procedure = pd.read_csv("Data/patient_procedure events.csv")
pat_postop = pd.read_csv("Data/patient_post_op_complications.csv")

## Postoperative complications

#### Variables to use
The **outcome** of interest is postoperative hypoxemia, which is recorded in `SMRTDTA_ELEM_VALUE`.

#### Wrangling procedures
`SMRTDTA_ELEM_VALUE` is a free text column, need to trasform into a *binary* target column:
1. Convert to all to lower case
2. Positive outcome (hpyoxemia): cells containing the keyword "hypoxemia"
3. Negative outcome (no hypoxemia): all other complications
    - Include NAs in `SMRTDTA_ELEM_VALUE` (which should represent no complication)

In [7]:
pat_postop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203945 entries, 0 to 203944
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   LOG_ID              203945 non-null  object
 1   MRN                 203945 non-null  object
 2   Element_Name        203945 non-null  object
 3   CONTEXT_NAME        203945 non-null  object
 4   Element_abbr        203945 non-null  object
 5   SMRTDTA_ELEM_VALUE  6741 non-null    object
dtypes: object(6)
memory usage: 9.3+ MB


In [8]:
pat_postop.head()

,LOG_ID,MRN,Element_Name,CONTEXT_NAME,Element_abbr,SMRTDTA_ELEM_VALUE
0,b00c588406aae5cc,1a42f0bed60c5598,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,NaN
1,a16571d21874c0a7,596a6dc60f54a0bb,AN AQI POST-OP COMPLICATIONS,NOTE,AN Post-op Complications,NaN
2,a16571d21874c0a7,596a6dc60f54a0bb,AN AQI POST-OP COMPLICATIONS,NOTE,AN Post-op Complications,NaN
3,1d9ff2d157fa4c86,60e6ea6cf06b58d7,AN AQI POST-OP COMPLICATIONS,ORDER,AN Post-op Complications,NaN
4,2ae43a4a39735956,59c5483e0654e105,AN AQI POST-OP COMPLICATIONS,ENCOUNTER,AN Post-op Complications,Other


## Patient information

#### Variables to use

- `BIRTH_DATE`: patients' age in years.
- `SEX`: genotypical sex of the patients.
- `HEIGHT`: height in feet and inches (ex. 5'6).
- `WEIGHT`: weight in ounces.
- `AN_START_DATETIME`, `AN_STOP_DATETIME`: date and time at which anesthesia was started / stopped.

#### Wrangling procedures

**Rename** `BIRTH_DATE` to `AGE` for clarity.

**Unit / type Conversion**

1. Convert `HEIGHT` into centimeters so the colum is numeric.
    - Use `df['HEIGHT'].str.split("'", expand=TRUE).astype(float)` to split `HEIGHT` into two columns (feet and inches, `expand=True` creates a separate df for this).
    - Conversion formula $\text{height in cm} = ((\text{feet} \times 12) + \text{inches}) \times 2.54$
2. (Optional) Convert `WEIGHT` into kg (or grams) for consistency.
    - $\text{weight in kg} = \text{ounces} \times 0.0283495$
3. Convert `AN_START_DATETIME` and `AN_STOP_DATETIME` to `datetime`.

**Handle missing values**
1. No missings for `BIRTH_DATE` and `SEX`.
2. Missing values in `HEIGHT` and `WEIGHT` could be imputed (ex. by regression imputation using other demographic features as predictors).


In [9]:
pat_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65728 entries, 0 to 65727
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   LOG_ID                65728 non-null  object 
 1   MRN                   65728 non-null  object 
 2   DISCH_DISP_C          65721 non-null  float64
 3   DISCH_DISP            65721 non-null  object 
 4   HOSP_ADMSN_TIME       65728 non-null  object 
 5   HOSP_DISCH_TIME       65714 non-null  object 
 6   LOS                   65714 non-null  float64
 7   ICU_ADMIN_FLAG        65728 non-null  object 
 8   SURGERY_DATE          65728 non-null  object 
 9   BIRTH_DATE            65728 non-null  int64  
 10  HEIGHT                52695 non-null  object 
 11  WEIGHT                63319 non-null  float64
 12  SEX                   65728 non-null  object 
 13  PRIMARY_ANES_TYPE_NM  65168 non-null  object 
 14  ASA_RATING_C          58758 non-null  float64
 15  ASA_RATING         

In [17]:
cols_of_interest = ["LOG_ID", "MRN", "BIRTH_DATE", "SEX", "HEIGHT", "WEIGHT", 
                    "AN_START_DATETIME", "AN_STOP_DATETIME"]
pat_demo = pat_info[cols_of_interest]
pat_demo.head()

,LOG_ID,MRN,BIRTH_DATE,SEX,HEIGHT,WEIGHT,AN_START_DATETIME,AN_STOP_DATETIME
0,cd1636c6279d73a0,2e92522a5f2a2a22,47,Female,NaN,2832.47,12/20/18 12:27,12/20/18 17:34
1,fc53c06ee2cfe438,b7a91e623d957d8f,81,Male,5' 6,2515.01,9/27/19 9:52,9/27/19 11:44
2,2f0090c47da34608,b68d6c7198f30f73,49,Female,NaN,1964.74,4/8/19 14:41,4/8/19 18:53
3,6d3b621880c6ced4,46aa48e89b9c50e8,54,Female,5' 3,2638.47,12/3/18 8:09,12/3/18 11:52
4,82f89e10f2855420,c2756216cc89b795,44,Female,NaN,1880.08,4/19/19 8:14,4/19/19 15:00


In [11]:
pat_demo['HEIGHT'].str.split("'", expand=True).astype(float)

,0,1
0,NaN,NaN
1,5.0,6.0
2,NaN,NaN
3,5.0,3.0
4,NaN,NaN
...,...,...
65723,5.0,9.0
65724,5.0,3.0
65725,NaN,NaN
65726,5.0,4.0


## Labs

#### Variables to use

- `Lab Code`: Logical Observation Identifier Names and Codes (LOINC) code for the laboratory test.
- `Lab Name`: long common name for the laboratory test.
- `Observation Value`: observed value for the lab.
- `Measurement Units`: units associated with observed value.
- `Collection Datetime`: date and time at which the lab was taken.
    - Used to related lab measurements to `EVENT_TIME` in the `patient procedure events` data.
 
#### Wrangling procedures
**Unit / type Conversion**
1. Convert `Collection Datetime` to `datetime`.


In [12]:
pat_labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29079344 entries, 0 to 29079343
Data columns (total 10 columns):
 #   Column               Dtype  
---  ------               -----  
 0   LOG_ID               object 
 1   MRN                  object 
 2   ENC_TYPE_NM          object 
 3   Lab Code             object 
 4   Lab Name             object 
 5   Observation Value    float64
 6   Measurement Units    object 
 7   Reference Range      object 
 8   Abnormal Flag        object 
 9   Collection Datetime  object 
dtypes: float64(1), object(9)
memory usage: 2.2+ GB


In [13]:
pat_labs.head()

,LOG_ID,MRN,ENC_TYPE_NM,Lab Code,Lab Name,Observation Value,Measurement Units,Reference Range,Abnormal Flag,Collection Datetime
0,0da0bc3be9423462,1bb09d5761661c7d,Hospital Encounter,11475-1,Microorganism identified,9999999.0,Unknown,Unknown,N,2021-01-14 20:58:00
1,0da0bc3be9423462,1bb09d5761661c7d,Hospital Encounter,1975-2,Bilirubin,1.4,mg/dL,0.0-1.4,N,2021-01-13 10:41:00
2,3ab8e6344cc9f954,559c869f9d7db8ee,Hospital Encounter,19235-1,Base excess^^standard,9999999.0,MMOL/L,Unknown,N,2020-03-08 00:40:00
3,3ab8e6344cc9f954,559c869f9d7db8ee,Hospital Encounter,19235-1,Base excess^^standard,9999999.0,MMOL/L,Unknown,N,2020-03-10 13:30:00
4,3ab8e6344cc9f954,559c869f9d7db8ee,Hospital Encounter,1959-6,Bicarbonate,20.0,MMOL/L,21-27,L,2020-03-07 23:33:00


## Procedure events

#### Variables to use
- `EVENT_DISPLAY_NAME`: name of the anesthesia event.
- `EVENT_TIME`: date and time at which the event occurred.

#### Wrangling procedure
1. Convert `EVENT_TIME` to `datetime`.

In [14]:
pat_procedure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640223 entries, 0 to 640222
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   LOG_ID              640223 non-null  object
 1   MRN                 640223 non-null  object
 2   EVENT_DISPLAY_NAME  640223 non-null  object
 3   EVENT_TIME          640223 non-null  object
 4   NOTE_TEXT           210 non-null     object
dtypes: object(5)
memory usage: 24.4+ MB


In [16]:
pat_procedure.head(10)

,LOG_ID,MRN,EVENT_DISPLAY_NAME,EVENT_TIME,NOTE_TEXT
0,a739a81fc152e77c,3936fad285669edf,"Transported to PACU/ICU with O2, vital signs s...",8/9/19 11:47,NaN
1,a6494a6557e211ff,3b095ccb1c4f124f,Two Anti-Emetics Administered,6/17/19 11:54,NaN
2,a6494a6557e211ff,3b095ccb1c4f124f,Two Anti-Emetics Administered,6/17/19 11:54,NaN
3,e1a066d449ed18b0,499386c1a8693fd0,Sign In,4/27/19 0:38,NaN
4,e1a066d449ed18b0,499386c1a8693fd0,Quick Note,4/27/19 3:04,NaN
5,022870bf6507aaee,e51fc26f0e7729a1,Sign In,5/1/19 7:18,NaN
6,fb943d65e18f9037,5b63201d6d190849,Sign In,9/27/19 7:19,NaN
7,744c92ba33fe09a6,184e84422ba4505c,Sign In,5/5/19 7:21,NaN
8,fee03b07e3b9e67d,06ee1994483efbb6,MAC Initiation,7/15/19 8:08,NaN
9,068b548cf0d91976,68d58e4c2dcd06f6,Quick Note,5/6/19 8:38,NaN


## Data Merging

To answer the RQ, the final dataset needs to have the following variables:

- **Outcome**: binary indicator of postoperative hypoxemia (0 = no, 1 = yes)

- **Predictors**:

    - Demographic variables: BMI, sex and age
    - Latest available preoperative lab test results (time stamp, lab name and measurement (with units)), where we use the latest lab test results before the start of anesthesia. 

Merge `patient_post_op_complications` `patient_information`, `patient_labs` and `patient_procedure_events` by the unique identifiers `LOG_ID` and `MRN`. 

**Feature engineering**
1. Convert the outcome to binary 0/1.
2. Create `BMI` variable based on height and weight (`HEIGHT` and `WEIGHT` variables should have the desired units and with no missing values by this step).
3. Create time stamp for when the lastest available preoperative lab test occurred
    - For each encounter (unique `MRN, LOG_ID` combination), find only the most recent `Collection Datetime` (from `patient_labs`) before `AN_START_DATETIME` (from `patient_information`).
    - Retain also the corresponding lab test name and measurements for that specific test.
4. (Optional) Get the name for the specific anesthesia procedure by mapping `AN_START_DATETIME` to `EVENT_TIME` (from `patient_procedure_events`).
    